In [1]:
import numpy as np


class Audiobook_Data_Reader():
    
    def __init__(self, dataset, batch_size = None):
        npz = np.load('Audiobook_data_{0}.npz'.format(dataset))
        
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
     
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        return inputs_batch, targets_one_hot
    
    def __iter__(self):
        return self

# Create Machine Learning Algorithm

In [2]:
import tensorflow as tf

input_size = 10
output_size = 2
hidden_layer_size = 50

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32,[None, input_size])
targets = tf.placeholder(tf.int32,[None, output_size])

weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable('biases_1', [hidden_layer_size])

outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

weights_2 = tf.get_variable('weights_2', [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable('biases_2', [hidden_layer_size])

outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.get_variable('weights_3', [hidden_layer_size, hidden_layer_size])
biases_3 = tf.get_variable('biases_3', [hidden_layer_size])

outputs_3 = tf.nn.tanh(tf.matmul(outputs_2, weights_3) + biases_3)

weights_4 = tf.get_variable('weights_4', [hidden_layer_size, hidden_layer_size])
biases_4 = tf.get_variable('biases_4', [hidden_layer_size])

outputs_4 = tf.nn.tanh(tf.matmul(outputs_3, weights_4) + biases_4)

weights_5 = tf.get_variable('weights_5', [hidden_layer_size, output_size])
biases_5 = tf.get_variable('biases_5', [output_size])

outputs = tf.matmul(outputs_4, weights_5) + biases_5

loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=targets)
mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

out_equals_targets = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))

accuracy = tf.reduce_mean(tf.cast(out_equals_targets, tf.float32))

sess = tf.InteractiveSession()

initializer = tf.global_variables_initializer()

sess.run(initializer)

batch_size = 50

max_epochs = 50

prev_validation_loss = 9999999.

train_data = Audiobook_Data_Reader('train', batch_size)
validation_data = Audiobook_Data_Reader('validation')

for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.
    for input_batch, target_batch in train_data:
        _, batch_loss = sess.run([optimize, mean_loss], feed_dict={inputs:input_batch, targets:target_batch})
        curr_epoch_loss += batch_loss
    curr_epoch_loss /= train_data.batch_count
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
                                                        feed_dict={inputs: input_batch, targets: target_batch})
        print('Epoch '+ str(epoch_counter+1)+
         '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
         '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
         '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    if validation_loss > prev_validation_loss:
        break
    prev_validation_loss = validation_loss
print('End of training')

Epoch 1. Training loss: 0.470. Validation loss: 0.398. Validation accuracy: 76.96%
Epoch 2. Training loss: 0.366. Validation loss: 0.381. Validation accuracy: 78.97%
Epoch 3. Training loss: 0.346. Validation loss: 0.374. Validation accuracy: 79.19%
Epoch 4. Training loss: 0.335. Validation loss: 0.371. Validation accuracy: 79.64%
Epoch 5. Training loss: 0.329. Validation loss: 0.371. Validation accuracy: 79.87%
End of training


# Test the model

In [3]:
test_data = Audiobook_Data_Reader('test')

for input_batch, target_batch in test_data:
        test_accuracy = sess.run([accuracy], feed_dict={inputs: input_batch, targets: target_batch})
test_accuracy_percent = test_accuracy[0]*100.
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 82.59%
